In [ ]:
# Polygon mask functions from https://github.com/JuliaImages/ImageDraw.jl/issues/56

function point_in_polygon(poly_xs::Vector{T}, poly_ys::Vector{T}, x::T, y::T) where T<: Real
    n_verts = length(poly_xs)
    j = n_verts
    c = false
    for i in 1:n_verts
        if (((poly_ys[i] <= y) && (y < poly_ys[j])) || ((poly_ys[j] <= y) && (y < poly_ys[i]))) && 
            (x < (poly_xs[j] - poly_xs[i]) * (y - poly_ys[i]) / (poly_ys[j] - poly_ys[i]) + poly_xs[i])
            c = !c
        end
        j = i
    end
    return c
end

# Changed these two functions to use bool matrix instead of values
function draw_polygon!(mask::Matrix{Bool}, poly_xs::Vector{Int}, poly_ys::Vector{Int})
    min_x, max_x = max(minimum(poly_xs), 1), min(maximum(poly_xs), size(mask, 2))
    min_y, max_y = max(minimum(poly_ys), 1), min(maximum(poly_ys), size(mask, 1))
    for y in min_y:max_y
        for x in min_x:max_x
            if point_in_polygon(poly_xs, poly_ys, x, y)
                mask[y, x] = true
            end
        end
    end
end

function polygons_to_mask(polygons::Array{Matrix{T}, 1} where T <: Real, max_x::Int, max_y::Int)::Matrix{Bool}
    poly_mask = zeros(Bool, max_y, max_x)  # Initialize a Bool matrix with all values set to false
    for p in polygons
        draw_polygon!(poly_mask, round.(Int, p[:, 1]), round.(Int, p[:, 2]))
    end
    return poly_mask
end


In [ ]:
# Testing auto STIV w/ real output
STIV_vec_Canny_sq = STIV_vec_Canny[251:749,1:499]
plt.imshow(STIV_vec_Canny_sq)
plt.gcf()
size(STIV_vec_Canny_sq)

import Images
import LinearAlgebra
import FFTW
import PyPlot as plt  # Use plt for plotting

# Initialize variables
# y = zeros(Float64, length(x), length(x))
# y1 = zeros(Float64, length(x), length(x))
# y2 = zeros(Float64, length(x), length(x))

# Populate y1, y2, and y arrays
# for i in 1:length(x)
#     y1[i, :] = sin.(0.15 * (x .- i/2))
#     y2[i, :] = 0.9 .* sin.(0.05 * (x .- i))
#     y[i, :] = y1[i, :] .+ y2[i, :]
# end
y = STIV_vec_Canny_sq
x = 1:size(STIV_vec_Canny_sq)[1]
xCenter = 250
yCenter = 250
radius = 249

# Display the y array as an image
# plt.figure()
# plt.imshow(y)
# plt.title("y Array")
# plt.colorbar()

# Compute the 2D Fourier transform and the power spectrum
Rxy = FFTW.ifft(abs.(FFTW.fft(y)).^2)
plt.figure()
plt.imshow(real(log.(Rxy[2:end,:])))
plt.title("Rxy Power Spectrum")
plt.colorbar()

# Calculate absolute value of spectra to plot
Rxy_real = real(Rxy)

# # Create a polar array
Rxy_pol = zeros(Float64, length(x), length(x))

# Define center and radius for circular mask
# xCenter = 501
# yCenter = 501
# radius = 500


# Define angles and compute coordinates for the circle
# theta_c = range(0, stop=2*pi, length=round(Int, 4 * pi * radius))
θ_c = range(1, stop = 360, length = 360)
xc = radius * cosd.(θ_c) .+ xCenter
yc = radius * sind.(θ_c) .+ yCenter

# Create a circular mask
circ_mask = polygons_to_mask([hcat(xc, yc)], xCenter+radius, yCenter+radius)

circ_nan = ones(Float64, size(circ_mask))
circ_nan[.!circ_mask] .= NaN

θ = zeros(Float64, length(x), length(x))
ρ = zeros(Float64, length(x), length(x))
Tx = zeros(Float64, length(x), length(x))
Ty = zeros(Float64, length(x), length(x))
M = 15  # "coefficient of intensification"

# Calculate theta and ρ for each pixel within the mask
for i in 1:length(x)
    for j in 1:length(x)
        if circ_mask[i, j]
            Tx[i,j] = xCenter - j
            Ty[i,j] = yCenter - i
            θ[i,j] = atand(Ty[i,j] / Tx[i, j])
            ρ[i,j] = M * log(sqrt(Tx[i,j]^2 + Ty[i,j]^2))
        end
    end
end

# Apply the circular mask to Rxy
Rxy_pol_norm = Rxy_real .* circ_nan ./ Rxy_real[yCenter, xCenter]
plt.figure()
plt.imshow(Rxy_pol_norm)
plt.title("Rxy Polar")
plt.colorbar()

ρ = ρ .* circ_nan
θ = θ .* circ_nan

# plt.figure()
# plt.imshow(θ)
# plt.title("Theta")
# plt.colorbar()

# plt.figure()
# plt.imshow(ρ)
# plt.title("Rho")
# plt.colorbar()

# Convert theta to degrees and calculate polar coordinates
# theta_deg = theta .* (180 / pi) .+ 90  # make all values positive
pol_x = exp.(ρ) .* cos.(θ)
pol_y = exp.(ρ) .* sin.(θ)

# plt.figure()
# plt.pcolormesh(ρ, theta_deg, Rxy_pol)
# plt.colorbar()
# plt.title("Polar Mesh")

# # Calculate the rounded rho array

# Aggregate by polar coordinates
# AngleIncrement = 0.1
AngleVec = [1*AngleIncrement:1*AngleIncrement:180]
θ_int = zeros(Float64, AngleVec)
inds_vec = zeros(Int, AngleVec)

for i in 1:AngleVec
    a = i-90
    inds = findall(θ -> round(θ) == a, θ[:])
    inds_vec[i] = length(inds)
    θ_int[i] = sum(Rxy_pol_norm[inds])
end

# plt.figure()
# plt.plot(inds_vec)
# plt.title("Indices Vector")

# plt.figure()
# plt.plot(θ_int)
# plt.title("Theta Integral")

# Rxy_norm = Rxy[xCenter, yCenter]
# θ_int_norm = θ_int ./ (Rxy_norm * Statistics.mean(inds_vec))

# plt.figure()
# plt.plot(θ_int_norm)
# plt.title("Normalized Theta Integral")


ρ_round = round.(ρ)
μ = zeros(Float64, degs)
ρ_max = M*log(minimum([maximum(filter(!isnan,Tx)),maximum(filter(!isnan,Ty))]))
for i in 1:degs
    a = i-90
    inds = findall(θ -> round(θ, digits = 1) == a, θ[:])
    Rxy_int = sum(Rxy_pol_norm[inds])
    μ[i] = (1/ρ_max)*Rxy_int
end

plt.figure()
plt.plot(μ)
plt.title("Directional Average Distribution")
plt.gcf()


In [ ]:
maximum(filter(!isnan, ρ))
ρ_round = round.(ρ)
ρ_round[200,200]
maximum(filter(!isnan, ρ_round))

In [ ]:
theta_c = range(1, stop = 360, length = 360)
xc = radius * cosd.(theta_c) .+ xCenter
yc = radius * sind.(theta_c) .+ yCenter

# Create a circular mask
circ_mask = zeros(Bool, 1001, 1001)
for i in 1:length(theta_c)
    circ_mask[round(Int, yc[i]), round(Int, xc[i])] = true
end



function point_in_polygon(poly_xs::Vector{T}, poly_ys::Vector{T}, x::T, y::T) where T<: Real
    n_verts = length(poly_xs)
    j = n_verts
    c = false
    for i in 1:n_verts
        if (((poly_ys[i] <= y) && (y < poly_ys[j])) || ((poly_ys[j] <= y) && (y < poly_ys[i]))) && 
            (x < (poly_xs[j] - poly_xs[i]) * (y - poly_ys[i]) / (poly_ys[j] - poly_ys[i]) + poly_xs[i])
            c = !c
        end
        j = i
    end
    return c
end

function draw_polygon!(mask::Matrix{T2}, poly_xs::Vector{T}, poly_ys::Vector{T}, value::T2) where T<: Integer where T2 <: Real
    min_x, max_x = max(minimum(poly_xs), 1), min(maximum(poly_xs), size(mask, 2))
    min_y, max_y = max(minimum(poly_ys), 1), min(maximum(poly_ys), size(mask, 1))
    for y in min_y:max_y
        for x in min_x:max_x
            if point_in_polygon(poly_xs, poly_ys, x, y)
                mask[y, x] = value
            end
        end
    end
end

function polygons_to_mask(polygons::Array{Matrix{T}, 1} where T <: Real, max_x::Int, max_y::Int)
    poly_mask = zeros(Int, max_y, max_x);
    for (i,p) in enumerate(polygons)
        draw_polygon!(poly_mask, round.(Int, p[:,1]), round.(Int, p[:,2]), i)
    end
    return poly_mask
end

circ_mask = polygons_to_mask([hcat(xc, yc)], 1001, 1001)


In [ ]:
plt.figure()
plt.pcolormesh(ρ, theta_deg, Rxy_pol)
plt.colorbar()
plt.title("Polar Mesh")

In [ ]:
deg_start = 1
deg_end = 180

for m in deg_start:deg_end
    # print(m)
    # inds = findall(θ -> round(Int, θ) == m, θ[:])
    # inds_vec[m] = length(inds)
    # θ_int[m] = sum(Rxy_pol[inds])
end
degs = 180
θ_int = zeros(Float64, degs)
# inds_vec = zeros(Int, degs)